In [1]:
# Check PyTorch version installed on this system
!python -c "import torch; print(torch.__version__)"

1.13.0+cu116


In [2]:
!nvidia-smi

Thu Dec 15 02:19:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    25W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Download the required modules
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.0+cu116
"""
%env TORCH=1.13.0+cu116
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install torchmetrics
!pip install matplotlib
!pip install networkx
!pip install scipy

env: TORCH=1.13.0+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
github_username=""
github_repository="grl"
github_token = ""

%env github_repository={github_repository}

env: github_repository=grl


In [5]:
!git clone https://{github_token}@github.com/{github_username}/{github_repository}.git

fatal: destination path 'grl' already exists and is not an empty directory.


In [6]:
%cd $github_repository

/content/grl


In [7]:
import torch
import random

random.seed(0)
torch.manual_seed(0)

In [8]:
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import AddSelfLoops
import dataset

train_dataset = PPI(root='', split='train', transform=AddSelfLoops())
val_dataset = PPI(root='', split='val', transform=AddSelfLoops())
test_dataset = PPI(root='', split='test', transform=AddSelfLoops())

dataset.num_features = 50
dataset.num_labels = 121
dataset.train_loader = DataLoader(train_dataset, batch_size=2)
dataset.val_loader = DataLoader(val_dataset, batch_size=2)
dataset.test_loader = DataLoader(test_dataset, batch_size=2)

In [9]:
from evaluation import evaluate
from training_loop import train

In [10]:
from models import GCNPPI
model = GCNPPI(dataset.num_features, dataset.num_labels)
evaluate(model, dataset.test_loader)

(tensor(0.6931, device='cuda:0'), tensor(0.3787, device='cuda:0'))

In [11]:
ppi_train_params = {
  "lr": 5e-3,
  "weight_decay": 0,
  "epochs": 400,
  "patience": 100,
  "model_name": model.model_name
}

In [12]:
def run_experiment(iters=10):
  losses = []
  scores = []
  for iter in range(iters):
    best_model, _, _, _, _, _ = train(model, ppi_train_params, verbose=False)
    loss, score = evaluate(best_model, dataset.test_loader)
    losses.append(loss)
    scores.append(score)
  losses = torch.tensor(losses)
  scores = torch.tensor(scores)
  return (torch.std_mean(losses), torch.std_mean(scores))

In [13]:
loss_ci, score_ci = run_experiment(10)

training model GCNPPI
GCNPPI(
  (encoder): Sequential(
    (0): GCNConv(50, 1024)
    (1): GCNConv(1024, 1024)
  )
  (classifier): GCNConv(1024, 121)
)
training...
early stopping...
best model performance @ epoch 00010: 
	val_loss: 0.6163 | val_micro_f1: 0.4974
	test_loss: 0.6061 | test_micro_f1: 0.5062
training model GCNPPI
GCNPPI(
  (encoder): Sequential(
    (0): GCNConv(50, 1024)
    (1): GCNConv(1024, 1024)
  )
  (classifier): GCNConv(1024, 121)
)
training...
early stopping...
best model performance @ epoch 00008: 
	val_loss: 0.6167 | val_micro_f1: 0.4960
	test_loss: 0.6063 | test_micro_f1: 0.5054
training model GCNPPI
GCNPPI(
  (encoder): Sequential(
    (0): GCNConv(50, 1024)
    (1): GCNConv(1024, 1024)
  )
  (classifier): GCNConv(1024, 121)
)
training...
early stopping...
best model performance @ epoch 00009: 
	val_loss: 0.6165 | val_micro_f1: 0.4971
	test_loss: 0.6052 | test_micro_f1: 0.5072
training model GCNPPI
GCNPPI(
  (encoder): Sequential(
    (0): GCNConv(50, 1024)
   

In [14]:
loss_std, loss_mean = loss_ci
score_std, score_mean = score_ci

print(f'loss:\t\t{loss_mean:.4f} +/- {loss_std:.4f}')
print(f'micro F1 score: {score_mean:.4f} +/- {score_std:.4f}')

loss:		0.6050 +/- 0.0008
micro F1 score: 0.5067 +/- 0.0010
